In [ ]:
import os
from openai import OpenAI

client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])

#Replace this directory path with the path to the directory of the RAG papers
assistant_data = "/home/adrian/Documents/University Work/Part III Project/cmbagent_dataset/Source_Papers"

chunking_strategy =  {
        "type": "static",
        "static": {
            "max_chunk_size_tokens": 4000, # reduce size to ensure better context integrity
            "chunk_overlap_tokens": 100 # increase overlap to maintain context across chunks
        }}
    
vector_store = client.vector_stores.create(name="rag_eval_test", chunking_strategy=chunking_strategy)
file_paths = []
for root, dirs, files in os.walk(assistant_data):
    # Filter out unwanted directories like .ipynb_checkpoints
    dirs[:] = [d for d in dirs if not d.startswith('.')]
    for file in files:
        if file.startswith('.') or file.endswith('.ipynb')  or file.endswith('.yaml') or file.endswith('.txt') or (not '.' in file):
            continue
        file_paths.append(os.path.join(root, file))
    
file_streams = [open(path, "rb") for path in file_paths]
    
file_batch = client.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id,
        files=file_streams
        )

#print vector_store.id to use in OpenAI RAG code
print(vector_store.id)